In [1]:
import sqlite3
import datetime as dt
import re
from dotenv import load_dotenv
from textblob import TextBlob
import nltk
import pandas as pd
from dateutil import parser
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [5]:
dt.datetime.now().strftime("%m-%d-%Y %H:%M:%S")

'03-15-2022 00:07:11'

In [18]:
df = pd.read_csv('PoliticalTexts.csv', index_col='ROWID')
# df['text'] = [x.replace(reciever_name, '<recipient name>', 5) for x in df['text']]
df['text'] = [x.replace('corey', '<recipient name>', 5) for x in df['text']]

In [19]:
df['timestamp'] = pd.to_datetime(df['date'], infer_datetime_format=True) + pd.offsets.DateOffset(years=31)
df['text'] = df['text'].replace('\\n', ' ')
df['month_name'] = df['timestamp'].dt.month_name()
df['day_name'] = df['timestamp'].dt.day_name()
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['weekday'] = df['timestamp'].dt.weekday
df['week'] = df['timestamp'].dt.isocalendar().week
df['year'] = df['timestamp'].dt.isocalendar().year

In [20]:
indices = [134844,134845,135206,135207,135264,135265,138476,
                   138477,138478,138479,138480,138482,138483,142979,
                   142980,144198,144199,144200]
for i in indices:
    df = df.drop(index=i)

In [21]:
df.head()

,SenderPhoneNumber,Sender,text,subject,country,attributedBody,type,date,date_delivered,timestamp,month_name,day_name,day,hour,weekday,week,year
ROWID,,,,,,,,,,,,,,,,,
130705,43367,Biden,Are our records correct?\n2020 total: $0\n2020...,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,621871120999425024,0,2020-09-15 13:58:40.999425024,September,Tuesday,15,13,1,38,2020
130731,43367,Biden,"DNC HQ: How will you vote?\nReply A, B, or C:\...",NaN,NaN, streamtyped���@���NSMutableAttributedString,0,621961369756412032,0,2020-09-16 15:02:49.756412032,September,Wednesday,16,15,2,38,2020
130733,43367,Biden,B.,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,621967223542932992,0,2020-09-16 16:40:23.542932992,September,Wednesday,16,16,2,38,2020
130734,43367,Biden,Great! We will reach out by text to let you kn...,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,621967225853177984,0,2020-09-16 16:40:25.853177984,September,Wednesday,16,16,2,38,2020
130939,43367,Biden,DNC HQ: McConnell has promised to fill this Su...,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,622314893951938048,0,2020-09-20 17:14:53.951938048,September,Sunday,20,17,6,38,2020


In [6]:
blob = TextBlob(df['text'].iloc[0])
blob.noun_phrases
blob.tags
blob.sentiment
len(blob.split())

27

In [7]:
len(blob.tokenize())

36

In [8]:
blob.sentiment_assessments

Sentiment(polarity=0.19318181818181818, subjectivity=0.32727272727272727, assessments=[(['new'], 0.13636363636363635, 0.45454545454545453, None), (['legal', '!'], 0.25, 0.2, None)])

In [9]:
blob.sentiment.polarity

0.19318181818181818

In [10]:
analyzer.polarity_scores(df['text'].iloc[0])

{'neg': 0.0, 'neu': 0.683, 'pos': 0.317, 'compound': 0.9032}

In [22]:
df['polarity'] = [TextBlob(sms).sentiment.polarity for sms in df['text']]
df['subjectivity'] = [TextBlob(sms).sentiment.subjectivity for sms in df['text']]
df['negativity'] = [analyzer.polarity_scores(sms)['neg'] for sms in df['text']]
df['neutrality'] = [analyzer.polarity_scores(sms)['neu'] for sms in df['text']]
df['positivity'] = [analyzer.polarity_scores(sms)['pos'] for sms in df['text']]
df['compound'] = [analyzer.polarity_scores(sms)['compound'] for sms in df['text']]
df['nouns'] = [TextBlob(sms).noun_phrases for sms in df['text']]
df['tags'] = [TextBlob(sms).tags for sms in df['text']]
df['word_count'] = [len(TextBlob(sms).split()) for sms in df['text']]

In [23]:
nouns = []

for text in df['nouns']:
    for noun in text:
        if noun == 'corey':
            nouns.append('<recipient name>')
        elif noun == 'stop2end':
            pass 
        else:
            nouns.append(noun)
pd.Series(nouns).value_counts()

trump                 354
pres trump            325
<recipient name>      297
donate                255
you                   183
                     ... 
petition                1
response requested      1
quick note              1
only official           1
failed                  1
Length: 1563, dtype: int64

In [24]:
impact_parser = re.compile('(\d\d\d.(?<=%)|\d\d\d\d.(?<=%))|(\dX(?<=X))', re.IGNORECASE)



test = []
for text in df['text']:
    if 'impact' in text.lower():
        test.append(text)

In [25]:
for text in test:
    m = impact_parser.match(text)
    if m:
        print(m.group())

1000%
400%
400%


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1057 entries, 130705 to 156486
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   SenderPhoneNumber  1057 non-null   int64         
 1   Sender             1057 non-null   object        
 2   text               1057 non-null   object        
 3   subject            1 non-null      object        
 4   country            0 non-null      float64       
 5   attributedBody     1057 non-null   object        
 6   type               1057 non-null   int64         
 7   date               1057 non-null   int64         
 8   date_delivered     1057 non-null   int64         
 9   timestamp          1057 non-null   datetime64[ns]
 10  month_name         1057 non-null   object        
 11  day_name           1057 non-null   object        
 12  day                1057 non-null   int64         
 13  hour               1057 non-null   int64         
 14  w

In [28]:
df.head(10)

,SenderPhoneNumber,Sender,text,subject,country,attributedBody,type,date,date_delivered,timestamp,...,year,polarity,subjectivity,negativity,neutrality,positivity,compound,nouns,tags,word_count
ROWID,,,,,,,,,,,,,,,,,,,,,
130705,43367,Biden,Are our records correct?\n2020 total: $0\n2020...,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,621871120999425024,0,2020-09-15 13:58:40.999425024,...,2020,0.000000,0.750000,0.000,1.000,0.000,0.0000,"[e-day, donate, dems, finish line]","[(Are, NNP), (our, PRP$), (records, NNS), (cor...",24
130731,43367,Biden,"DNC HQ: How will you vote?\nReply A, B, or C:\...",NaN,NaN, streamtyped���@���NSMutableAttributedString,0,621961369756412032,0,2020-09-16 15:02:49.756412032,...,2020,0.000000,0.000000,0.095,0.905,0.000,-0.4466,"[dnc hq, reply, a. vote-by-mail, vote, c., vot...","[(DNC, NNP), (HQ, NNP), (How, WRB), (will, MD)...",29
130733,43367,Biden,B.,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,621967223542932992,0,2020-09-16 16:40:23.542932992,...,2020,0.000000,0.000000,0.000,1.000,0.000,0.0000,[],"[(B, NNP)]",1
130734,43367,Biden,Great! We will reach out by text to let you kn...,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,621967225853177984,0,2020-09-16 16:40:25.853177984,...,2020,0.500000,0.483333,0.000,0.766,0.234,0.6696,[place info],"[(Great, NN), (We, PRP), (will, MD), (reach, V...",20
130939,43367,Biden,DNC HQ: McConnell has promised to fill this Su...,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,622314893951938048,0,2020-09-20 17:14:53.951938048,...,2020,0.100000,0.400000,0.000,0.661,0.339,0.8500,"[dnc hq, mcconnell, supreme court seat, join, ...","[(DNC, NNP), (HQ, NNP), (McConnell, NNP), (has...",24
131210,43367,Biden,￼DNC HQ: Did you read President Obama's email?...,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,622939067142160000,0,2020-09-27 22:37:47.142160000,...,2020,0.216667,0.444444,0.000,0.911,0.089,0.5423,"[hq, obama, 's email, unity, 's end-of-quarter...","[(￼DNC, JJ), (HQ, NNP), (Did, NNP), (you, PRP)...",48
131294,43367,Biden,DNC HQ: Democrats can't win without you.\n\nNa...,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,623173015774612992,0,2020-09-30 15:36:55.774612992,...,2020,0.350000,0.494444,0.109,0.891,0.000,-0.4717,"[dnc hq, ca n't, name, donate, midnight deadline]","[(DNC, NNP), (HQ, NNP), (Democrats, NNPS), (ca...",26
131327,43367,Biden,Don't miss out! Join Pres. Obama for a special...,NaN,NaN, streamtyped���@���NSMutableAttributedString,0,623258205414608896,0,2020-10-01 15:16:45.414608896,...,2020,0.446429,0.571429,0.064,0.777,0.160,0.4291,"[join pres, obama, special event, et, donate, ...","[(Do, VBP), (n't, RB), (miss, VB), (out, RP), ...",26
131457,43367,Biden,"DNC HQ: Since you haven't donated this year, w...",NaN,NaN, streamtyped���@���NSMutableAttributedString,0,623531448001283072,0,2020-10-04 19:10:48.001283072,...,2020,0.050000,0.200000,0.000,0.954,0.046,0.0772,[dnc hq],"[(DNC, NNP), (HQ, NNP), (Since, IN), (you, PRP...",28


In [26]:
df.to_csv('ProcessedTexts.csv', index='ROWID')